<a href="https://colab.research.google.com/github/remisoulignac/scm_optim_problems/blob/main/SC3X_Scream2_Hyper_parameter_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective
Refer to SC3X-Scream1_MILP.ipynb for a presentation of the problem.

The online simulator is here : https://scxscream.herokuapp.com/home/
It is only accessible during the challenges.

#Solution approach
As explained in the V1 approach of the solution, it was difficult to modelize with MILP the online SCREAM simulator (in its version v4) and identify clearly which objective function we tried to optimize.

In this V2 approach :

1) we don't try to replicate the SCREAM simulator. We use its engine online by calling its API to do the calculations. Then, we just need to find the best combinaison of imput parameters to get the optimal solution.

2) we use a simplified way to construct the objective function. It is now a weighted average based on probability of occurence of various failure modes. The scenario is static and prebuilt.

To find the optimal parameters, we use the the hyper-parameter optimization library [hyperopt](http://hyperopt.github.io/hyperopt/). We pass it an objective function (the online REST API), the search space for the parameters, and voilà :) 

As a prerequesite to run this notebook, the online challenge must be open, its API accessible and compliant with th expected schema.

Moreover, the goal of this notebook is only to showcase on approach using hyper-parameter optimization technique and not to give the answer. Hence, the scenario values have been tampered.


In [ ]:
!pip install requests
!pip install hyperopt

In [ ]:
import numpy as np

API_URL="http://scxscream.herokuapp.com/tests/"

#to get these information by pressing F11 on your web browser in the online SCREAM simulator and inspect the calls to the API "test"
COOKIE = "csrftoken=xxxx; sessionid=yyyy"
XCSRFToken = "zzzz"

def check_scenario(scenario):
  proba_sum = np.array(scenario).sum(axis=0)[6]
  print(f"sum of probabilities : {proba_sum}")
  assert(abs(proba_sum-1.0)<0.001)

  #even number of lines
  assert (len(scenario)%2==0)

  return scenario



# need an even number of scenario because with push 2 scenarii at the same time to the API
# the _S values are week number when the  facility starts to be off
# the _D values are the duration of the failure
# note that the details of the scenario was tampered
scenario2=check_scenario([
#  ["DC_S", "DC_D", "Pl_S", "Pl_D", "Su_S", "Su_D", "Prob"]          
   [     1,      0,      1,      0,      1,      0,   0.70], #Failure Mode 1
   [     1,      0,      1,      0,      1,      0,   0.03], #Failure Mode 2
   [     1,      0,      1,      0,      1,      0,   0.03], #Failure Mode 3
   [     1,      0,      1,      0,      1,      0,   0.03], #Failure Mode 4
   [     1,      0,      1,      0,      1,      0,   0.03], #Failure Mode 5
   [     1,      0,      1,      0,      1,      0,   0.03], #Failure Mode 6
   [     1,      0,      1,      0,      1,      0,   0.03], #Failure Mode 7
   [     1,      0,      1,      0,      1,      0,   0.06], #Failure Mode 8
   [     1,      0,      1,      0,      1,      0,   0.05], #Failure Mode 9
   [     1,      0,      1,      0,      1,      0,   0.01], #Failure Mode 10
])

def find_best_solution(scenario, max_evals=10, penalty_missed_items=10):
  check_scenario(scenario)
  from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
  import requests

  def objective(hyperparameters):
    print(hyperparameters)
    avgTotalProfit=0.0
    avgIfrActual=0.0
    it=iter(scenario)
    for sc1, sc2 in zip(it, it): #zip by two
      response = requests.get(API_URL)
      payload={
          "altDisruptionsOne":
                {
                    "disruptionKey":"testkey",
                    "dcStart":"{:.2f}".format(52 if sc1[1]==0 else sc1[0]),
                    "dcDuration": (max(sc1[1],1)), 
                    "plantStart":"{:.2f}".format(52 if sc1[3]==0 else sc1[2]),
                    "plantDuration":(max(sc1[3],1)),
                    "supplierStart":"{:.2f}".format(52 if sc1[5]==0 else sc1[4]),
                    "supplierDuration":(max(sc1[5],1)),
                },
            "altDisruptionsTwo":
                {
                    "disruptionKey":"testkey",
                    "dcStart":"{:.2f}".format(52 if sc2[1]==0 else sc2[0]),
                    "dcDuration": (max(sc2[1],1)), 
                    "plantStart":"{:.2f}".format(52 if sc2[3]==0 else sc2[2]),
                    "plantDuration":(max(sc2[3],1)),
                    "supplierStart":"{:.2f}".format(52 if sc2[5]==0 else sc2[4]),
                    "supplierDuration":(max(sc2[5],1)), 
                },
            "userInputOne":
              {
                  "fgi":hyperparameters["fgi"]*10,
                  "wip":hyperparameters["wip"]*10,
                  "waitToStart":"true",
                  "supplier":str(hyperparameters["supplier"]),
                  "plant":str(hyperparameters["plant"]),
                  "dc":str(hyperparameters["dc"])
              },
            "userInputTwo":
              {
                  "fgi":hyperparameters["fgi"]*10,
                  "wip":hyperparameters["wip"]*10,
                  "waitToStart":"true",
                  "supplier":str(hyperparameters["supplier"]),
                  "plant":str(hyperparameters["plant"]),
                  "dc":str(hyperparameters["dc"])
              }
          }
      headers={
          "Cookie" : COOKIE,
          "Content-type": "application/json",
          "X-CSRFToken": XCSRFToken
        }
      raw_response = requests.post(API_URL, headers=headers, json = payload)
      response=raw_response.json()
      # {'out1': {'altDisruptionsOne': {'avgIfrActual': 0.04215,
      #   'avgTotalCosts': 14330.77,
      #  'avgTotalProfit': 15639.23,
      #  'avgTotalRevenue': 29970.0}},
      #'out2': {'altDisruptionsTwo': {'avgIfrActual': 0.04215,
      #  'avgTotalCosts': 14330.77,
      #  'avgTotalProfit': 15639.23,
      #  'avgTotalRevenue': 29970.0}}}
      avgTotalProfit = avgTotalProfit+response["out1"]["altDisruptionsOne"]["avgTotalProfit"]*sc1[6]
      avgIfrActual = avgIfrActual+response["out1"]["altDisruptionsOne"]["avgIfrActual"]*sc1[6]
      avgTotalProfit = avgTotalProfit+response["out2"]["altDisruptionsTwo"]["avgTotalProfit"]*sc2[6]
      avgIfrActual = avgIfrActual+response["out2"]["altDisruptionsTwo"]["avgIfrActual"]*sc2[6]
    
    loss= -(avgTotalProfit- (1-avgIfrActual)*5200*penalty_missed_items)
    print(f"loss:{loss}, avgTotalProfit:{avgTotalProfit}, avgIfrActual:{avgIfrActual}")
    return {
        'loss': loss,
        'status':STATUS_OK,
        'attachments':  {"avgTotalProfit": avgTotalProfit, "avgIfrActual" : avgIfrActual }
      }

  search_space = {
      'fgi': hp.randint('fgi', 520),
      'wip': hp.randint('wip', 520),
      'dc' : hp.choice('dc', [1, 2, 3, 4, 5, 6, 7]),
      'plant' : hp.choice('plant', [1, 2, 3, 4, 5, 6, 7]),
      'supplier' : hp.choice('supplier', [1, 2, 3, 4, 5, 6, 7])
  }

  trials = Trials()
  best = fmin(objective,
      space=search_space,
      algo=tpe.suggest,
      max_evals=max_evals,
      trials=trials)
  
  return trials

sum of probabilities : 1.0000000000000002


In [ ]:
sol2 = find_best_solution(scenario2, 1000)

In [ ]:
print(sol2.best_trial["misc"]["vals"])
print(sol2.trial_attachments(sol2.best_trial)['avgTotalProfit'])
print(sol2.trial_attachments(sol2.best_trial)['avgIfrActual'])